In [5]:
# Domains of the variables
domains = {
    'A': [1, 2, 3, 4],
    'B': [1, 2, 3, 4, 5],
    'C': [1, 2, 3, 4, 5],
    'D': [1, 2, 3, 4, 5]
}

# Constraints as lambda functions, these return True if the constraint is satisfied
constraints = {
    ('A', 'B'): lambda A, B: A > B,
    ('B', 'C'): lambda B, C: B > C,
    ('C', 'D'): lambda C, D: C > D
}

# Function to get the least constraining value
def get_least_constraining_value(var, assignment):
    if not assignment:
        return sorted(domains[var])
    possible_values = []
    for value in domains[var]:
        is_least_constraining = True
        for other_var in assignment:
            if (var, other_var) in constraints:
                if not constraints[(var, other_var)](value, assignment[other_var]):
                    is_least_constraining = False
                    break
            elif (other_var, var) in constraints:
                if not constraints[(other_var, var)](assignment[other_var], value):
                    is_least_constraining = False
                    break
        if is_least_constraining:
            possible_values.append(value)
    return sorted(possible_values)

# Function to select the unassigned variable using the MRV and MCV heuristics
def select_unassigned_variable(assignment):
    unassigned_vars = [v for v in domains if v not in assignment]
    unassigned_vars.sort(key=lambda var: (len(domains(var, assignment)), -sum(1 for constraint in constraints if var in constraint), var))
    return unassigned_vars[0] if unassigned_vars else None

# The main backtracking function with path tracking
def backtrack(assignment, path=[]):
    if len(assignment) == len(domains):
        return assignment
    var = select_unassigned_variable(assignment)
    for value in get_least_constraining_value(var, assignment):
        new_assignment = assignment.copy()
        new_assignment[var] = value
        new_path = path + [(var, value)]  # Update the path with the current assignment
        result = backtrack(new_assignment, new_path)
        if result is not None:
            return result
    # if not path:  # Only print if there's something in the path (i.e., not at the root call)
    print("Backtracking from:", path)
    return None

# Start the backtracking process and print each branch upon backtracking
solution = backtrack({})

if solution:
    print("Solution found:", solution)
else:
    print("No solution exists.")


TypeError: 'dict' object is not callable

In [26]:
def load_variables(filepath):
    domains = {}
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(':')
            variable = parts[0].strip()
            values = [int(x) for x in parts[1].strip().split()]
            domains[variable] = values
    return domains

def load_constraints(filepath):
    constraints = []
    operators = ['=', '!', '>', '<']  # Define possible operators
    with open(filepath, 'r') as file:
        for line in file:
            op_index = -1
            op_char = ''
            for op in operators:  # Find the operator in the line
                op_index = line.find(op)
                if op_index != -1:
                    op_char = op
                    break
            if op_index != -1:  # If an operator was found
                var1 = line[:op_index].strip()
                var2 = line[op_index+1:].strip()
                constraints.append((var1, op_char, var2))
    return constraints



# Example usage
domains = load_variables('../tests/ex4.var')
constraints = load_constraints('../tests/ex4.con')

    
def make_graph(domains, constraints):
    graph = {var: set() for var in domains}
    for var1, op, var2 in constraints:
        graph[var1].add(var2)
        graph[var2].add(var1)  # Assuming constraints are bidirectional
    return graph

graph = make_graph(domains, constraints)
print(graph)

numToNode = {i: var for i, var in enumerate(domains)}

def evaluate_constraint(var1, val1, var2, val2, op):
    if op == '=':
        return val1 == val2
    elif op == '!':
        return val1 != val2
    elif op == '>':
        return val1 > val2
    elif op == '<':
        return val1 < val2
    else:
        raise ValueError(f"Unknown operator: {op}")
    
def pairwise_constraints_check(var1, value1, var2, value2, constraints):
    for constraint in constraints:
        if constraint[0] == var1 and constraint[2] == var2:
            if not evaluate_constraint(var1, value1, var2, value2, constraint[1]):
                result = False
            else:
                result = True
        elif constraint[2] == var1 and constraint[0] == var2:
            if not evaluate_constraint(var2, value2, var1, value1, constraint[1]):
                result = False
            else:
                result = True
    return result  # If no constraint directly applies, the pair is considered to not violate any constraints


def constraints_check(assignment, constraints):
    for var1, op, var2 in constraints:
        if var1 in assignment and var2 in assignment:
            if not evaluate_constraint(var1, assignment[var1], var2, assignment[var2], op):
                return False
    return True

# Function to get the most constrained variable
def get_variable(assignment):
    unassigned_vars = [v for v in domains if v not in assignment]
    # Sort based on the number of remaining values in the domain, then by the variable that constrains others the most, and finally alphabetically
    unassigned_vars.sort(key=lambda var: (
        len(domains[var]),  # Fewest legal values first
        -len([other_var for other_var in graph[var] if other_var not in assignment]),  # Most constraining variable# Most constraining variable
        var                 # Alphabetically
    ))
    return unassigned_vars[0]



# # Function to get the least constraining values for a given variable
def get_values(var, assignment, tried_values):
    # Initialize a dictionary to keep track of how many options each value eliminates for each unassigned neighbor
    elimination_counts = {value: 0 for value in domains[var] if value not in tried_values[var]}
    connected_vars = graph[var]

    for value in elimination_counts.keys():
        for other_var in connected_vars:
            if other_var not in assignment:  # Consider only unassigned neighbors
                for other_value in domains[other_var]:
                    # Check if the current value would eliminate the other_value as an option
                    if not pairwise_constraints_check(var, value, other_var, other_value, constraints):
                        elimination_counts[value] += 1

    # Sort the values by the ones that eliminate the fewest options for others, breaking ties by the value itself
    sorted_values = sorted(elimination_counts, key=lambda x: (elimination_counts[x], x))
    return sorted_values

def print_formatted(assignment, condition):
    result = ""
    count = 1
    for val, node in enumerate(assignment):
        result += node
        result += '='
        result += str(val)
        if count < len(assignment):
            result += ', '
            count += 1
        elif condition:
            result += ' failure'
        else:
            result += ' solution'
    print(result)

# Backtrack function to find the solution
def backtrack(assignment, branches, tried_values):
    # Check if assignment is complete
    if len(assignment) == len(domains):
        return assignment, branches, tried_values

    # Get the next variable to assign
    var = get_variable(assignment)
    # Try all values for this variable
    for value in get_values(var, assignment, tried_values):
        new_assignment = assignment.copy()
        new_assignment[var] = value
        tried_values[var].append(value)

        # If the assignment is consistent, continue
        if constraints_check(new_assignment, constraints):
            result, branches, tried_values = backtrack(new_assignment, branches, tried_values)
            if result is not None:
                if len(new_assignment) == len(domains):
                    branches.append(f"{new_assignment} solution")
                    print_formatted(new_assignment, False)
                return result, branches, tried_values
        else:
            branches.append(f"{new_assignment} failure")
            print_formatted(new_assignment, True)

    # If no assignment was successful, backtrack
    tried_values[var] = []
    return None, branches, tried_values

# Initialize the backtrack algorithm
assignment = {}
branches = []
tried_values = {var: [] for var in domains}
solution, branches_visited, _ = backtrack(assignment, branches, tried_values)
# print(len(branches_visited))

print(domains)

{'A': {'B'}, 'B': {'C', 'A'}, 'C': {'D', 'B'}, 'D': {'C'}}
A=0, C=1, B=2 failure
A=0, C=1, B=2, D=3 failure
A=0, C=1, B=2, D=3 failure
A=0, C=1, B=2, D=3 failure
A=0, C=1, B=2, D=3 failure
A=0, C=1, B=2, D=3 failure
A=0, C=1, B=2, D=3 failure
A=0, C=1, B=2, D=3 failure
A=0, C=1, B=2, D=3 failure
A=0, C=1, B=2, D=3 failure
A=0, C=1, B=2, D=3 failure
A=0, C=1, B=2 failure
A=0, C=1, B=2 failure
A=0, C=1, B=2 failure
A=0, C=1, B=2 failure
A=0, C=1, B=2, D=3 solution
{'A': [1, 2, 3, 4], 'B': [1, 2, 3, 4, 5], 'C': [1, 2, 3, 4, 5], 'D': [1, 2, 3, 4, 5]}


In [4]:
import numpy as np

# Define the domains for each variable
domains = {
    'A': [1, 2, 3, 4],
    'B': [1, 2, 3, 4, 5],
    'C': [1, 2, 3, 4, 5],
    'D': [1, 2, 3, 4, 5]
}

graph = {'A': [0, 1, 0, 0],
         'B': [1, 0, 1, 0],
         'C': [0, 1, 0, 1],
         'D': [0, 0, 1, 0]}

numToNode = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D'
}

# Define the constraints as a function
def constraints_check(assignment):
    if 'A' in assignment and 'B' in assignment and not assignment['A'] > assignment['B']:
        return False
    elif 'B' in assignment and 'C' in assignment and not assignment['B'] > assignment['C']:
        return False
    elif 'C' in assignment and 'D' in assignment and not assignment['C'] > assignment['D']:
        return False
    return True

# Function to get the most constrained variable
def get_variable(assignment):
    unassigned_vars = [v for v in domains if v not in assignment]
    # Sort based on the number of remaining values in the domain, then by the variable that constrains others the most, and finally alphabetically
    unassigned_vars.sort(key=lambda var: (
        len(domains[var]),
        -sum(1 for idx, other_var in enumerate(graph[var]) if other_var == 1 and numToNode[idx] not in assignment),  # most constraining variable
        var
    ))
    return unassigned_vars[0]

def pairwise_constraints_check(var1, value1, var2, value2):
    result = True
    if var1 == 'A' and var2 == 'B':
        result = value1 > value2
    elif var1 == 'B' and var2 == 'C':
        result = value1 > value2
    elif var1 == 'C' and var2 == 'D':
        result = value1 > value2

    # Here we check for the reverse as well because the order of variables might be reversed.
    # if var2 == 'A' and var1 == 'B':
    #     result = value2 > value1
    # elif var2 == 'B' and var1 == 'C':
    #     result = value2 > value1
    # elif var2 == 'C' and var1 == 'D':
    #     result = value2 > value1
    
    # print(f"Checking constraint {var1} > {var2} with values {value1}, {value2}: {'Passed' if result else 'Failed'}")
    return result



# Function to get the least constraining values for a given variable
def get_values(var, assignment, tried_values):
    possible_values = list(set(domains[var]) - set(tried_values[var]))
    # Count how many values are eliminated for other variables for each value
    counts = {value: 0 for value in possible_values}
    # Get the list of variables connected to 'var'
    connected_vars = [numToNode[idx] for idx, connected in enumerate(graph[var]) if connected == 1]


    for value in possible_values:
        # print(f"Calculating LCV for {var}={value}")
        for other_var in connected_vars:
            if other_var not in assignment:
                for other_value in domains[other_var]:
                    if not pairwise_constraints_check(var, value, other_var, other_value):
                        counts[value] += 1
                        # print(f"Incrementing count for {var}={value} because of constraint with {other_var}={other_value}")
    # print(f"LCV Counts for {var}: {counts}")

    # Sort by the number of eliminated values, and numerically as a tie-breaker
    values = list(counts.keys())
    if var == 'C':
        print(counts)
    values.sort(key=lambda val: (counts[val], val))
    return values

# Backtrack function to find the solution
def backtrack(assignment, branches, tried_values):
    # Check if assignment is complete
    if len(assignment) == len(domains):
        
        return assignment, branches, tried_values

    # Get the next variable to assign
    var = get_variable(assignment)
    # Try all values for this variable
    for value in get_values(var, assignment, tried_values):
        new_assignment = assignment.copy()
        new_assignment[var] = value
        tried_values[var].append(value)

        # If the assignment is consistent, continue
        if constraints_check(new_assignment):
            result, branches, tried_values = backtrack(new_assignment, branches, tried_values)
            if result is not None:
                if len(new_assignment) == len(domains):
                    branches.append(f"{new_assignment} & S")
                return result, branches, tried_values
        else:
            branches.append(f"{new_assignment} & F")

    # If no assignment was successful, backtrack
    tried_values[var] = [];
    return None, branches, tried_values

# Initialize the backtrack algorithm
assignment = {}
branches = []
tried_values = {
    'A': [],
    'B': [],
    'C': [],
    'D': []
}
solution, branches_visited, _ = backtrack(assignment, branches, tried_values)
print(len(branches_visited))

branches_visited


{1: 5, 2: 4, 3: 3, 4: 2, 5: 1}
18


["{'A': 4, 'C': 5, 'B': 1} & F",
 "{'A': 4, 'C': 5, 'B': 2} & F",
 "{'A': 4, 'C': 5, 'B': 3} & F",
 "{'A': 4, 'C': 5, 'B': 4} & F",
 "{'A': 4, 'C': 5, 'B': 5} & F",
 "{'A': 4, 'C': 4, 'B': 1} & F",
 "{'A': 4, 'C': 4, 'B': 2} & F",
 "{'A': 4, 'C': 4, 'B': 3} & F",
 "{'A': 4, 'C': 4, 'B': 4} & F",
 "{'A': 4, 'C': 4, 'B': 5} & F",
 "{'A': 4, 'C': 3, 'B': 1} & F",
 "{'A': 4, 'C': 3, 'B': 2} & F",
 "{'A': 4, 'C': 3, 'B': 3} & F",
 "{'A': 4, 'C': 3, 'B': 4} & F",
 "{'A': 4, 'C': 3, 'B': 5} & F",
 "{'A': 4, 'C': 2, 'B': 1} & F",
 "{'A': 4, 'C': 2, 'B': 2} & F",
 "{'A': 4, 'C': 2, 'B': 3, 'D': 1} & S"]

In [ ]:
candidates = [(var, len(domains[var]) - sum(1 for value in domains[var] if is_value_consistent(var, value, assignment))) 
                  for var in domains if var not in assignment]